In [7]:
"""
see: https://github.com/BLarzalere/LSTM-Autoencoder-for-Anomaly-Detection/blob/master/Sensor%20Anomaly%20Detection.ipynb
"""

import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.externals import joblib
import seaborn as sns
sns.set(color_codes=True)
import matplotlib.pyplot as plt
%matplotlib inline
from io import StringIO

from numpy.random import seed
import tensorflow as tf

from tensorflow.keras.layers import Input, Dropout, Dense, LSTM, TimeDistributed, RepeatVector
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers

from google.cloud import storage

seed = 100

In [31]:
#data_dir = "gs://amazing-public-data/bearing_sensor_data/bearing_sensor_data/"
BUCKET_NAME = "amazing-public-data"
merged_data = pd.DataFrame()

client = storage.Client()
prefix = "bearing_sensor_data/bearing_sensor_data/"
blobs = client.list_blobs(BUCKET_NAME, prefix=prefix)

files = []
for blob in blobs:
    dataset = pd.read_csv("gs://{0}/{1}".format(BUCKET_NAME, blob.name), sep='\t')
    dataset_mean_abs = np.array(dataset.abs().mean())
    dataset_mean_abs = pd.DataFrame(dataset_mean_abs.reshape(1, 4))
    dataset_mean_abs.index = [blob.name.split("/")[-1]]
    merged_data = merged_data.append(dataset_mean_abs)
    files.append(blob.name)

merged_data.columns = ['bearing-1', 'bearing-2', 'bearing-3', 'bearing-4']
print(len(files))

982


In [32]:
merged_data.head()

,0,1,2,3
2004.02.12.10.52.39,0.060236,0.074227,0.083926,0.044443
2004.02.12.11.02.39,0.061455,0.073844,0.084457,0.045081
2004.02.12.11.12.39,0.061361,0.075609,0.082837,0.045118
2004.02.12.11.22.39,0.061665,0.073279,0.084879,0.044172
2004.02.12.11.32.39,0.061944,0.074593,0.082626,0.044659


In [34]:
# transform data file index to datetime and sort in chronological order
merged_data.index = pd.to_datetime(merged_data.index, format='%Y.%m.%d.%H.%M.%S')
merged_data = merged_data.sort_index()
print("Dataset shape:", merged_data.shape)
merged_data.head()

Dataset shape: (982, 4)


,0,1,2,3
2004-02-12 10:52:39,0.060236,0.074227,0.083926,0.044443
2004-02-12 11:02:39,0.061455,0.073844,0.084457,0.045081
2004-02-12 11:12:39,0.061361,0.075609,0.082837,0.045118
2004-02-12 11:22:39,0.061665,0.073279,0.084879,0.044172
2004-02-12 11:32:39,0.061944,0.074593,0.082626,0.044659


In [36]:
# drop the raw_data into a bucket
from io import StringIO
DEST_FILE_NAME = "raw_data.csv"
DEST_BUCKET_NAME = "rrusson-kubeflow-test"
f = StringIO()
merged_data.to_csv(f)
f.seek(0)
client.get_bucket(DEST_BUCKET_NAME).blob(DEST_FILE_NAME).upload_from_file(f, content_type='text/csv')

In [37]:
!(gcloud config get-value core/project)

mwpmltr
